In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import argparse
import os
import torch
import random
import numpy as np
import pandas as pd

os.chdir('/content/drive/MyDrive/work_space/Project/BI_Competition/BI_Project4_with_DLinear')
from exp.exp_main import Exp_Main
from utils2.Date_input import DateFilter, apply_pca, WeightedAverageCalculator

In [5]:
weight_file_path = './data/지역별_주거용전력판매량.csv'
raw_data_path = './data/weather_electric_data_2.csv'

In [7]:
calculator = WeightedAverageCalculator(weight_file_path, raw_data_path, '연도')
result_df = calculator.calculate_weighted_average()

In [8]:
date_filter = DateFilter(result_df, date_column='date')
filtered_df = date_filter.filter_by_date('2021-03-13', '2023-03-12 23:55:00') # 2년가정

columns_to_keep = ['date', '현재수요(MW)'] + [col for col in filtered_df.columns if '가중평균' in col]
columns_to_drop = [col for col in filtered_df.columns if col not in columns_to_keep]


filtered_df.drop(columns_to_drop, axis=1, inplace=True)
filtered_df.reset_index(inplace=True, drop=True)
filtered_df.to_csv('./data/temp_data.csv', index=False)

In [9]:
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [19]:
torch.cuda.is_available()

False

In [13]:
args = argparse.Namespace(
    # basic config
    is_training = 1,                                  # default = 1
    train_only = False,                               # default = False
    model_id = 'D_Linear_Test_1',
    model = 'DLinear',                                # options: [DLinear, NLinear, Autoformer, Informer, Transformer, etc...]

    # Data Loader
    data = 'custom',                                  # I dont know ... Maybe Name?
    root_path = './data/',                         #### Data Folder Path
    data_path = 'temp_data.csv',      #### Data File Path
    features = 'M',                                #### Forecating task, options: [M, S, MS], M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate.
    target ='현재수요(MW)',                          #### target feature in S or MS task. Its maybe name of target Columns

    # Need to check 'freq' works well.
    freq = '5min',                                 #### freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h.
    checkpoints = './checkpoints/',                   # location of model checkpoints (Need it?)

    # Forecating Task
    seq_len = (288*3),                                  #### Input Sequence Length.
    label_len = 72,                                #### Start Token Length.
    pred_len = 72,                                 #### Prediction Sequence Length.

    # DLinear
    individual = False,                               # DLinear: a linear layer for each variate(channel) individually (I dont know)

    # Formers
    embed_type = 1,                                   # 0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding.
    enc_in = 10,                                 #### Encoder Input Size. (I dont know why we need to control this?)
    dec_in = 7,                                       # Decoder Input Size.
    c_out = 7,                                        # Output size.
    d_model = 512,                                    # Dimension of model(size of model)
    n_heads = 8,                                      # Num of attention heads.
    e_layers = 2,                                     # Num of Encoder layers.
    d_layers = 1,                                     # Num of Decoder layers.
    d_ff = 2048,                                      # Dimension of fcn(Feed-Forward Network?)
    moving_avg = 31,                                  # Windoe size of moving average.
    factor = 1,                                       # Attention Factor.
    distil = True,                                    # Whether to use distilling in encoder, using this argument means not using distilling.
    dropout = 0.05,                                   # Dropout rate.
    embed = 'timeF',                                  # Time features encoding, options:[timeF, fixed, learned]
    activation = 'gelu',                              # Activation Function.
    output_attention = False,                         # Whether to output attention in ecoder.
    do_predict = True,                                # Whether to predict unseen future data.

    # Optimization
    num_workers = 10,                             ##### data loader num workers
    itr = 3,                                      ##### experiments times
    train_epochs = 5,                            ##### train epochs
    batch_size = 16,                              ##### batch size of train input data
    patience = 3,                                     # early stopping patience
    learning_rate = 0.001,                       ##### optimizer learning rate
    des = 'Exp',                                  ##### exp description (Almost 'Exp', maybe we dont need to change it)
    loss = 'mse',                                     # loss function
    lradj = 'type1',                                  # adjust learning rate
    use_amp = False,                                  # use automatic mixed precision training

    # GPU Setting
    use_gpu = True,                                   # Use GPU?
    gpu = 0,                                          # GPU
    use_multi_gpu = False,                            # Use Multiple GPUs? (r u reach guy?)
    devices = '0,1,2,3',                              # Device Indexs of multile GPUs.
    test_flop = False                                 # See utils/tools for usage ... I dont wanna check
)

In [ ]:
# Arguments Check

args

Namespace(is_training=1, train_only=False, model_id='D_Linear_Test_1', model='DLinear', data='custom', root_path='./data/', data_path='temp_df.csv', features='M', target='현재수요(MW)', freq='5min', checkpoints='./checkpoints/', seq_len=864, label_len=72, pred_len=72, individual=False, embed_type=0, enc_in=153, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, num_workers=10, itr=1, train_epochs=1, batch_size=16, patience=3, learning_rate=0.01, des='Exp', loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False)

In [14]:
# Model Run.

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Exp = Exp_Main

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        if not args.train_only:
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting)

        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)

        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                  args.model,
                                                                                                  args.data,
                                                                                                  args.features,
                                                                                                  args.seq_len,
                                                                                                  args.label_len,
                                                                                                  args.pred_len,
                                                                                                  args.d_model,
                                                                                                  args.n_heads,
                                                                                                  args.e_layers,
                                                                                                  args.d_layers,
                                                                                                  args.d_ff,
                                                                                                  args.factor,
                                                                                                  args.embed,
                                                                                                  args.distil,
                                                                                                  args.des, ii)

    exp = Exp(args)  # set experiments

    if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)
    else:
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting, test=1)
    torch.cuda.empty_cache()


Args in experiment:
Namespace(is_training=1, train_only=False, model_id='D_Linear_Test_1', model='DLinear', data='custom', root_path='./data/', data_path='temp_data.csv', features='M', target='현재수요(MW)', freq='5min', checkpoints='./checkpoints/', seq_len=864, label_len=72, pred_len=72, individual=False, embed_type=1, enc_in=10, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=31, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, num_workers=10, itr=3, train_epochs=5, batch_size=16, patience=3, learning_rate=0.001, des='Exp', loss='mse', lradj='type1', use_amp=False, use_gpu=False, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False)
Use CPU
>>>>>>>start training : D_Linear_Test_1_DLinear_custom_ftM_sl864_ll72_pl72_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 146233
val 20953
test 41977
	iters: 100, epoch: 1 | loss: 0.1617973
	speed: 0.016